<a href="https://colab.research.google.com/github/UncleStotheh/dl_healthcare_BIOF/blob/master/Week3_4_HW_SH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#I decided to follow the tutorial for the Age Regression model using MR images
#the scripts used were gathered from here: 
#https://github.com/DLTK/DLTK/tree/master/examples/applications/IXI_HH_age_regression_resnet

#i originally tried using my local computer to download and train the algorithm
#but i was running into several issues with simply downloading the data
#I managed to resolve the main issue which was that the downloading script was 
  ##downloading incomplete data (potentially corrupted) 
#Next I tried training the algorithm on my local computer, but it took a long time (low GPU + limited resources)

#TLDR: I copied my local jupyter notebook to google collab and decided to use the provided trained algorithm

#here are my past attempts and output once I used the pre-trained algorithm 

#collab attempt:
#I ran into issues since a module called tensorflow.contrib is no longer available (even after downgrading tensorflow)
#I had to find the original source code for various tensorflow.contrib command/packages
#hence why there are so many scripts here 
#i downloaded the demographic.csv file independently and uploaded from my local computer
#due to very slow and tedious downloading/processing of data
#i downloaded only the files for sample IXI566 locally and uploaded from my local computer
#the age regression output is found below

#if you have any questions let me know




In [51]:
#the file downloading script does not work completely....so I downloaded
#them manually and ran the script after downloading
#import wget
#url = "http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar"
#t2 = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T2.tar'
#pd = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-PD.tar'
#mra = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-MRA.tar'
#demographic = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI.xls'

#use an earlier version of tensorflow

###use lines between -----------------
#!pip uninstall tensorflow -y
#!pip install "tensorflow-gpu==1.4.0"
#%tensorflow_version 1.4

#!git clone https://github.com/DLTK/DLTK.git 

#import tensorflow as tf
#print(tf.__version__) # for both Python 2 and Python 3


#get pretrained model 
!wget "http://www.doc.ic.ac.uk/~mrajchl/dltk_models/examples/applications/IXI_HH_age_regression.tar.gz"
!tar -xvf '/content/IXI_HH_age_regression.tar.gz'


## use  lines between ------------------

#wget.download(t2, "/Users/hernandezs4/Downloads/Jupyter")
#wget.download(pd, "/Users/hernandezs4/Downloads/Jupyter")
#wget.download(mra, "/Users/hernandezs4/Downloads/Jupyter")
#wget.download(demographic, "/Users/hernandezs4/Downloads/Jupyter")

#changed 


2.3.0
--2020-10-11 20:44:54--  http://www.doc.ic.ac.uk/~mrajchl/dltk_models/examples/applications/IXI_HH_age_regression.tar.gz
Resolving www.doc.ic.ac.uk (www.doc.ic.ac.uk)... 146.169.13.6
Connecting to www.doc.ic.ac.uk (www.doc.ic.ac.uk)|146.169.13.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146092814 (139M) [application/x-gzip]
Saving to: ‘IXI_HH_age_regression.tar.gz.1’

IXI_HH_age_regressi 100%[===================>] 139.32M  9.69MB/s    in 16s     

2020-10-11 20:45:10 (8.90 MB/s) - ‘IXI_HH_age_regression.tar.gz.1’ saved [146092814/146092814]

IXI_HH_age_regression/
IXI_HH_age_regression/model.ckpt-50404.meta
IXI_HH_age_regression/.nfs0000000003096d1000000002
IXI_HH_age_regression/checkpoint
IXI_HH_age_regression/graph.pbtxt
IXI_HH_age_regression/model.ckpt-50404.index
IXI_HH_age_regression/1510857419/
IXI_HH_age_regression/1510857419/variables/
IXI_HH_age_regression/1510857419/variables/variables.data-00000-of-00001
IXI_HH_age_regression/1510857419/v

In [ ]:
#to download data 
# -*- coding: utf-8 -*-
"""Download and extract the IXI Hammersmith Hospital 3T dataset
url: http://brain-development.org/ixi-dataset/
ref: IXI – Information eXtraction from Images (EPSRC GR/S21533/02)
"""
from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
from future.standard_library import install_aliases  # py 2/3 compatability
install_aliases()

from urllib.request import FancyURLopener

import os.path
import tarfile
import pandas as pd
import glob
import SimpleITK as sitk
import numpy as np

DOWNLOAD_IMAGES = True
EXTRACT_IMAGES = True
PROCESS_OTHER = True
RESAMPLE_IMAGES = True
CLEAN_UP = True


def resample_image(itk_image, out_spacing=(1.0, 1.0, 1.0), is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
                int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
                int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)


def reslice_image(itk_image, itk_ref, is_label=False):
    resample = sitk.ResampleImageFilter()
    resample.SetReferenceImage(itk_ref)

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)


urls = {}
urls['t1'] = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T1.tar'
urls['t2'] = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-T2.tar'
urls['pd'] = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-PD.tar'
urls['mra'] = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI-MRA.tar'
urls['demographic'] = 'http://biomedic.doc.ic.ac.uk/brain-development/downloads/IXI/IXI.xls'

fnames = {}
fnames['t1'] = 't1.tar'
fnames['t2'] = 't2.tar'
fnames['pd'] = 'pd.tar'
fnames['mra'] = 'mra.tar'
fnames['demographic'] = 'demographic.xls'


if DOWNLOAD_IMAGES:
    # Download all IXI data
    for key, url in urls.items():

        if not os.path.isfile(fnames[key]):
            print('Downloading {} from {}'.format(fnames[key], url))
            curr_file = FancyURLopener()
            curr_file.retrieve(url, fnames[key])
        else:
            print('File {} already exists. Skipping download.'.format(
                fnames[key]))

if EXTRACT_IMAGES:
    # Extract the HH subset of IXI
    for key, fname in fnames.items():

        if (fname.endswith('.tar')):
            print('Extracting IXI HH data from {}.'.format(fnames[key]))
            output_dir = os.path.join('./orig/', key)

            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            t = tarfile.open(fname, 'r')
            for member in t.getmembers():
                if '-HH-' in member.name:
                    t.extract(member, output_dir)


if PROCESS_OTHER:
    # Process the demographic xls data and save to csv
    xls = pd.ExcelFile('demographic.xls')
    print(xls.sheet_names)

    df = xls.parse('Table')
    for index, row in df.iterrows():
        IXI_id = 'IXI{:03d}'.format(row['IXI_ID'])
        df.loc[index, 'IXI_ID'] = IXI_id

        t1_exists = len(glob.glob('./orig/t1/{}*.nii.gz'.format(IXI_id)))
        t2_exists = len(glob.glob('./orig/t2/{}*.nii.gz'.format(IXI_id)))
        pd_exists = len(glob.glob('./orig/pd/{}*.nii.gz'.format(IXI_id)))
        mra_exists = len(glob.glob('./orig/mra/{}*.nii.gz'.format(IXI_id)))

        # Check if each entry is complete and drop if not
        # if not t1_exists and not t2_exists and not pd_exists and not mra
        # exists:
        if not (t1_exists and t2_exists and pd_exists and mra_exists):
            df.drop(index, inplace=True)

    # Write to csv file
    df.to_csv('demographic_HH.csv', index=False)

if RESAMPLE_IMAGES:
    # Resample the IXI HH T2 images to 1mm isotropic and reslice all
    # others to it
    #had to change to .to_numpy
    df = pd.read_csv('demographic_HH.csv', dtype=object, keep_default_na=False,
                     na_values=[]).to_numpy()

    for i in df:
        IXI_id = i[0]
        print('Resampling {}'.format(IXI_id))

        t1_fn = glob.glob('./orig/t1/{}*.nii.gz'.format(IXI_id))[0]
        t2_fn = glob.glob('./orig/t2/{}*.nii.gz'.format(IXI_id))[0]
        pd_fn = glob.glob('./orig/pd/{}*.nii.gz'.format(IXI_id))[0]
        mra_fn = glob.glob('./orig/mra/{}*.nii.gz'.format(IXI_id))[0]

        t1 = sitk.ReadImage(t1_fn)
        t2 = sitk.ReadImage(t2_fn)
        pd = sitk.ReadImage(pd_fn)
        mra = sitk.ReadImage(mra_fn)

        # Resample to 1mm isotropic resolution
        t2_1mm = resample_image(t2)
        t1_1mm = reslice_image(t1, t2_1mm)
        pd_1mm = reslice_image(pd, t2_1mm)
        mra_1mm = reslice_image(mra, t2_1mm)

        output_dir = os.path.join('./1mm/', IXI_id)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print('T1: {} {}'.format(t1_1mm.GetSize(), t1_1mm.GetSpacing()))
        print('T2: {} {}'.format(t2_1mm.GetSize(), t2_1mm.GetSpacing()))
        print('PD: {} {}'.format(pd_1mm.GetSize(), pd_1mm.GetSpacing()))
        print('MRA: {} {}'.format(mra_1mm.GetSize(), mra_1mm.GetSpacing()))

        sitk.WriteImage(t1_1mm, os.path.join(output_dir, 'T1_1mm.nii.gz'))
        sitk.WriteImage(t2_1mm, os.path.join(output_dir, 'T2_1mm.nii.gz'))
        sitk.WriteImage(pd_1mm, os.path.join(output_dir, 'PD_1mm.nii.gz'))
        sitk.WriteImage(mra_1mm, os.path.join(output_dir, 'MRA_1mm.nii.gz'))

        # Resample to 2mm isotropic resolution
        t2_2mm = resample_image(t2, out_spacing=[2.0, 2.0, 2.0])
        t1_2mm = reslice_image(t1, t2_2mm)
        pd_2mm = reslice_image(pd, t2_2mm)
        mra_2mm = reslice_image(mra, t2_2mm)

        output_dir = os.path.join('./2mm/', IXI_id)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print('T1: {} {}'.format(t2_2mm.GetSize(), t1_2mm.GetSpacing()))
        print('T2: {} {}'.format(t2_2mm.GetSize(), t2_2mm.GetSpacing()))
        print('PD: {} {}'.format(pd_2mm.GetSize(), pd_2mm.GetSpacing()))
        print('MRA: {} {}'.format(mra_2mm.GetSize(), mra_2mm.GetSpacing()))

        sitk.WriteImage(t1_2mm, os.path.join(output_dir, 'T1_2mm.nii.gz'))
        sitk.WriteImage(t2_2mm, os.path.join(output_dir, 'T2_2mm.nii.gz'))
        sitk.WriteImage(pd_2mm, os.path.join(output_dir, 'PD_2mm.nii.gz'))
        sitk.WriteImage(mra_2mm, os.path.join(output_dir, 'MRA_2mm.nii.gz'))


if CLEAN_UP:
    # Remove the .tar files
    for key, fname in fnames.items():
        if (fname.endswith('.tar')):
            os.remove(fname)

    # Remove all data in original resolution
    os.system('rm -rf orig')


File t1.tar already exists. Skipping download.
File t2.tar already exists. Skipping download.
File pd.tar already exists. Skipping download.
File mra.tar already exists. Skipping download.
File demographic.xls already exists. Skipping download.
Extracting IXI HH data from t1.tar.
Extracting IXI HH data from t2.tar.
Extracting IXI HH data from pd.tar.
Extracting IXI HH data from mra.tar.
['Table', 'Ethnicity', 'Marital Status', 'Occupation', 'Qualification', 'Study Date']
Resampling IXI012
T1: (230, 230, 134) (1.0, 1.0, 1.0)
T2: (230, 230, 134) (1.0, 1.0, 1.0)
PD: (230, 230, 134) (1.0, 1.0, 1.0)
MRA: (230, 230, 134) (1.0, 1.0, 1.0)
T1: (115, 115, 67) (2.0, 2.0, 2.0)
T2: (115, 115, 67) (2.0, 2.0, 2.0)
PD: (115, 115, 67) (2.0, 2.0, 2.0)
MRA: (115, 115, 67) (2.0, 2.0, 2.0)
Resampling IXI013
T1: (230, 230, 139) (1.0, 1.0, 1.0)
T2: (230, 230, 139) (1.0, 1.0, 1.0)
PD: (230, 230, 139) (1.0, 1.0, 1.0)
MRA: (230, 230, 139) (1.0, 1.0, 1.0)
T1: (115, 115, 70) (2.0, 2.0, 2.0)
T2: (115, 115, 70) (2.

KeyboardInterrupt: 

In [ ]:
#training script to split dataset into training and validation set

In [60]:
#make custom reader script for dltk 
%%writefile reader.py


from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import SimpleITK as sitk
import tensorflow as tf
import os
import numpy as np

from dltk.io.augmentation import flip, extract_random_example_array
from dltk.io.preprocessing import whitening


def read_fn(file_references, mode, params=None):
    """A custom python read function for interfacing with nii image files.
    Args:
        file_references (list): A list of lists containing file references, such
            as [['id_0', 'image_filename_0', target_value_0], ...,
            ['id_N', 'image_filename_N', target_value_N]].
        mode (str): One of the tf.estimator.ModeKeys strings: TRAIN, EVAL or
            PREDICT.
        params (dict, optional): A dictionary to parametrise read_fn ouputs
            (e.g. reader_params = {'n_examples': 10, 'example_size':
            [64, 64, 64], 'extract_examples': True}, etc.).
    Yields:
        dict: A dictionary of reader outputs for dltk.io.abstract_reader.
    """

    def _augment(img):
        """An image augmentation function"""
        return flip(img, axis=2)

    for f in file_references:
        subject_id = f[0]

        data_path = '/content/IXI_HH_age_regression'

        # Read the image nii with sitk
        t1_fn = os.path.join(data_path, '{}/T1_2mm.nii.gz'.format(subject_id))
        t1 = sitk.GetArrayFromImage(sitk.ReadImage(str(t1_fn)))

        # Normalise volume image
        t1 = whitening(t1)

        # Create a 4D image (i.e. [x, y, z, channels])
        images = np.expand_dims(t1, axis=-1).astype(np.float32)

        if mode == tf.estimator.ModeKeys.PREDICT:
            yield {'features': {'x': images}, 'img_id': subject_id}

        # Parse the regression targets from the file_references
        age = np.float(f[11])
        y = np.expand_dims(age, axis=-1).astype(np.float32)

        # Augment if used in training mode
        if mode == tf.estimator.ModeKeys.TRAIN:
            images = _augment(images)

        # Check if the reader is supposed to return training examples or full
        #  images
        if params['extract_examples']:
            images = extract_random_example_array(
                image_list=images,
                example_size=params['example_size'],
                n_examples=params['n_examples'])

            for e in range(params['n_examples']):
                yield {'features': {'x': images[e].astype(np.float32)},
                       'labels': {'y': y.astype(np.float32)},
                       'img_id': subject_id}

        else:
            yield {'features': {'x': images},
                   'labels': {'y': y.astype(np.float32)},
                   'img_id': subject_id}

    return


Overwriting reader.py


In [2]:
#had to adjust the abstract_reader.py
%%writefile sh_abstract_reader.py

from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf
import traceback

#changed this line to tf.compat.v1.train.SessionRunHook
class IteratorInitializerHook(tf.compat.v1.train.SessionRunHook):
    """Hook to initialise data iterator after Session is created."""

    def __init__(self):
        super(IteratorInitializerHook, self).__init__()
        self.iterator_initializer_func = None

    def after_create_session(self, session, coord):
        """Initialise the iterator after the session has been created."""
        self.iterator_initializer_func(session)


class Reader(object):
    """Wrapper for dataset generation given a read function"""

    def __init__(self, read_fn, dtypes):
        """Constructs a Reader instance
        Args:
            read_fn: Input function returning features which is a dictionary of
                string feature name to `Tensor` or `SparseTensor`. If it
                returns a tuple, first item is extracted as features.
                Prediction continues until `input_fn` raises an end-of-input
                exception (`OutOfRangeError` or `StopIteration`).
            dtypes:  A nested structure of tf.DType objects corresponding to
                each component of an element yielded by generator.
        """
        self.dtypes = dtypes

        self.read_fn = read_fn

    def get_inputs(self,
                   file_references,
                   mode,
                   example_shapes=None,
                   shuffle_cache_size=100,
                   batch_size=4,
                   params=None):
        """
        Function to provide the input_fn for a tf.Estimator.
        Args:
            file_references: An array like structure that holds the reference
                to the file to read. It can also be None if not needed.
            mode: A tf.estimator.ModeKeys. It is passed on to `read_fn` to
                trigger specific functions there.
            example_shapes (optional): A nested structure of lists or tuples
                corresponding to the shape of each component of an element
                yielded by generator.
            shuffle_cache_size (int, optional): An `int` determining the
                number of examples that are held in the shuffle queue.
            batch_size (int, optional): An `int` specifying the number of
                examples returned in a batch.
            params (dict, optional): A `dict` passed on to the `read_fn`.
        Returns:
            function: a handle to the `input_fn` to be passed the relevant
                tf estimator functions.
            tf.train.SessionRunHook: A hook to initialize the queue within
                the dataset.
        """
        iterator_initializer_hook = IteratorInitializerHook()

        def train_inputs():
            def f():
                def clean_ex(ex, compare):
                    # Clean example dictionary by recursively deleting
                    # non-relevant entries. However, this does not look into
                    # dictionaries nested into lists
                    for k in list(ex.keys()):
                        if k not in list(compare.keys()):
                            del ex[k]
                        elif isinstance(ex[k], dict) and isinstance(compare[k], dict):
                            clean_ex(ex[k], compare[k])
                        elif (isinstance(ex[k], dict) and not isinstance(compare[k], dict)) or \
                             (not isinstance(ex[k], dict) and isinstance(compare[k], dict)):
                            raise ValueError('Entries between example and '
                                             'dtypes incompatible for key {}'
                                             ''.format(k))
                        elif (isinstance(ex[k], list) and not isinstance(compare[k], list)) or \
                            (not isinstance(ex[k], list) and isinstance(compare[k], list)) or \
                                (isinstance(ex[k], list) and isinstance(compare[k], list) and not
                                    len(ex[k]) == len(compare[k])):
                            raise ValueError('Entries between example and '
                                             'dtypes incompatible for key {}'
                                             ''.format(k))
                    for k in list(compare):
                        if k not in list(ex.keys()):
                            raise ValueError('Key {} not found in ex but is '
                                             'present in dtypes. Found keys: '
                                             '{}'.format(k, ex.keys()))
                    return ex

                fn = self.read_fn(file_references, mode, params)
                # iterate over all entries - this loop is terminated by the
                # tf.errors.OutOfRangeError or StopIteration thrown by the
                # read_fn
                while True:
                    try:
                        ex = next(fn)

                        if ex.get('labels') is None:
                            ex['labels'] = None

                        if not isinstance(ex, dict):
                            raise ValueError('The read_fn has to return '
                                             'dictionaries')

                        ex = clean_ex(ex, self.dtypes)
                        yield ex
                    except (tf.errors.OutOfRangeError, StopIteration):
                        raise
                    except Exception as e:
                        print('got error `{} from `_read_sample`:'.format(e))
                        print(traceback.format_exc())
                        raise

            dataset = tf.data.Dataset.from_generator(
                f, self.dtypes, example_shapes)
            dataset = dataset.repeat(None)
            dataset = dataset.shuffle(shuffle_cache_size)
            dataset = dataset.batch(batch_size)
            dataset = dataset.prefetch(1)

            iterator = dataset.make_initializable_iterator()
            next_dict = iterator.get_next()

            # Set runhook to initialize iterator
            iterator_initializer_hook.iterator_initializer_func = \
                lambda sess: sess.run(iterator.initializer)

            # Return batched (features, labels)
            return next_dict['features'], next_dict.get('labels')

        # Return function and hook
        return train_inputs, iterator_initializer_hook

    def serving_input_receiver_fn(self, placeholder_shapes):
        """Build the serving inputs.
        Args:
            placeholder_shapes: A nested structure of lists or tuples
                corresponding to the shape of each component of the feature
                elements yieled by the read_fn.
        Returns:
            function: A function to be passed to the tf.estimator.Estimator
            instance when exporting a saved model with estimator.export_savedmodel.
        """

        def f():
            inputs = {k: tf.placeholder(
                shape=[None] + list(placeholder_shapes['features'][k]),
                dtype=self.dtypes['features'][k]) for k in list(self.dtypes['features'].keys())}

            return tf.estimator.export.ServingInputReceiver(inputs, inputs)
        return f


Writing sh_abstract_reader.py


In [228]:
#had to edit the train.py script since it needed to access my edited version of 
#sh_abstract_reader.py


In [64]:
#here I tried to load the provided python scripts and edited scripts as modules
#!pip install dltk
import dltk  

#!git clone https://github.com/DLTK/DLTK.git
#for some reason the custom scripts even from dltk have to be in /content/

sh_abstract_reader_path = '/content/sh_abstract_reader.py'
path_to_read = "/content/reader.py"

import sys
import os
sys.path.append('/content/sh_abstract_reader.py')
sys.path.append(os.path.abspath(path_to_read))
import sh_abstract_reader
#from sh_abstract_reader import Reader
import reader


In [11]:
#edited version of train.py
#will output to a file named sh_train.py
%%writefile sh_train.py

# -*- coding: utf-8 -*-
from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import argparse
import os
import pandas as pd
import tensorflow as tf
#.compat.v1
#tf.disable_v2_behavior()
import numpy as np

from dltk.networks.regression_classification.resnet import resnet_3d
from sh_abstract_reader import Reader

import reader 

EVAL_EVERY_N_STEPS = 100
EVAL_STEPS = 5

NUM_CLASSES = 1
NUM_CHANNELS = 1

BATCH_SIZE = 8
SHUFFLE_CACHE_SIZE = 32

MAX_STEPS = 50000


def model_fn(features, labels, mode, params):
    """Model function to construct a tf.estimator.EstimatorSpec. It creates a
        network given input features (e.g. from a dltk.io.abstract_reader) and
        training targets (labels). Further, loss, optimiser, evaluation ops and
        custom tensorboard summary ops can be added. For additional information,
        please refer to
        https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator#model_fn.
    Args:
        features (tf.Tensor): Tensor of input features to train from. Required
            rank and dimensions are determined by the subsequent ops (i.e.
            the network).
        labels (tf.Tensor): Tensor of training targets or labels. Required rank
            and dimensions are determined by the network output.
        mode (str): One of the tf.estimator.ModeKeys: TRAIN, EVAL or PREDICT
        params (dict, optional): A dictionary to parameterise the model_fn
            (e.g. learning_rate)
    Returns:
        tf.estimator.EstimatorSpec: A custom EstimatorSpec for this experiment
    """

    # 1. create a model and its outputs
    net_output_ops = resnet_3d(
        inputs=features['x'],
        num_res_units=2,
        num_classes=NUM_CLASSES,
        filters=(16, 32, 64, 128, 256),
        strides=((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2)),
        mode=mode,
        kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-4))

    # 1.1 Generate predictions only (for `ModeKeys.PREDICT`)
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=net_output_ops,
            export_outputs={'out': tf.estimator.export.PredictOutput(net_output_ops)})

    # 2. set up a loss function
    loss = tf.losses.mean_squared_error(
        labels=labels['y'],
        predictions=net_output_ops['logits'])

    # 3. define a training op and ops for updating moving averages (i.e.
    # for batch normalisation)
    global_step = tf.train.get_global_step()
    optimiser = tf.train.AdamOptimizer(learning_rate = params["learning_rate"],
        epsilon=1e-5)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimiser.minimize(loss, global_step=global_step)

    # 4.1 (optional) create custom image summaries for tensorboard
    my_image_summaries = {}
    my_image_summaries['feat_t1'] = features['x'][0, 32, :, :, 0]

    expected_output_size = [1, 96, 96, 1]  # [B, W, H, C]
    [tf.summary.image(name, tf.reshape(image, expected_output_size))
     for name, image in my_image_summaries.items()]

    # 4.2 (optional) track the rmse (scaled back by 100, see reader.py)
    rmse = tf.metrics.root_mean_squared_error
    mae = tf.metrics.mean_absolute_error
    eval_metric_ops = {"rmse": rmse(labels['y'], net_output_ops['logits']),
                       "mae": mae(labels['y'], net_output_ops['logits'])}

    # 5. Return EstimatorSpec object
    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=net_output_ops,
                                      loss=loss,
                                      train_op=train_op,
                                      eval_metric_ops=eval_metric_ops)


def train(args):
    np.random.seed(42)
    tf.set_random_seed(42)

    print('Setting up...')

    # Parse csv files for file names
    all_filenames = pd.read_csv(
        args.data_csv,
        dtype=object,
        keep_default_na=False,
        na_values=[]).to_numpy()

    train_filenames = all_filenames[:150]
    val_filenames = all_filenames[150:]

    # Set up a data reader to handle the file i/o.
    reader_params = {'n_examples': 2,
                     'example_size': [64, 96, 96],
                     'extract_examples': True}

    reader_example_shapes = {'features': {'x': reader_params['example_size'] + [NUM_CHANNELS, ]},
                             'labels': {'y': [1]}}

    fixed = Reader(reader.read_fn, {'features': {'x': tf.float32},
                              'labels': {'y': tf.float32}})

    # Get input functions and queue initialisation hooks for training and
    # validation data

    train_input_fn, train_qinit_hook = fixed.get_inputs(
        file_references=train_filenames,
        mode=tf.estimator.ModeKeys.TRAIN,
        example_shapes=reader_example_shapes,
        batch_size=BATCH_SIZE,
        shuffle_cache_size=SHUFFLE_CACHE_SIZE,
        params=reader_params)

    val_input_fn, val_qinit_hook = fixed.get_inputs(
        file_references=val_filenames,
        mode=tf.estimator.ModeKeys.EVAL,
        example_shapes=reader_example_shapes,
        batch_size=BATCH_SIZE,
        shuffle_cache_size=SHUFFLE_CACHE_SIZE,
        params=reader_params)

    # Instantiate the neural network estimator
    nn = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir=args.model_path,
        params={"learning_rate": 0.001},
        config=tf.estimator.RunConfig())

    # Hooks for validation summaries
    val_summary_hook = tf.contrib.training.SummaryAtEndHook(
        os.path.join(args.model_path, 'eval'))
    step_cnt_hook = tf.train.StepCounterHook(
        every_n_steps=EVAL_EVERY_N_STEPS, output_dir=args.model_path)

    print('Starting training...')
    try:
        for _ in range(MAX_STEPS // EVAL_EVERY_N_STEPS):
            nn.train(input_fn=train_input_fn,
                     hooks=[train_qinit_hook, step_cnt_hook],
                     steps=EVAL_EVERY_N_STEPS)

            if args.run_validation:
                results_val = nn.evaluate(input_fn=val_input_fn,
                                          hooks=[val_qinit_hook, val_summary_hook],
                                          steps=EVAL_STEPS)
                print('Step = {}; val loss = {:.5f};'.format(
                    results_val['global_step'],
                    results_val['loss']))

    except KeyboardInterrupt:
        pass

    print('Stopping now.')

    # When exporting we set the expected input shape to be arbitrary.
    export_dir = nn.export_savedmodel(
        export_dir_base=args.model_path,
        serving_input_receiver_fn=reader.serving_input_receiver_fn(
            {'features': {'x': [None, None, None, NUM_CHANNELS]},
             'labels': {'y': [1]}}))
    print('Model saved to {}.'.format(export_dir))


if __name__ == '__main__':
    # Set up argument parser
    parser = argparse.ArgumentParser(description='Example: IXI HH resnet age regression training script')
    parser.add_argument('--run_validation', default=True)
    parser.add_argument('--restart', default=False, action='store_true')
    parser.add_argument('--verbose', default=False, action='store_true')
    parser.add_argument('--cuda_devices', '-c', default='0')

    parser.add_argument('--model_path', '-p', default='/tmp/IXI_age_regression/')
    parser.add_argument('--data_csv', default='../../../data/IXI_HH/demographic_HH.csv')

    args = parser.parse_args()

    # Set verbosity
    if args.verbose:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
        tf.logging.set_verbosity(tf.logging.INFO)
    else:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
        tf.logging.set_verbosity(tf.logging.ERROR)

    # GPU allocation options
    os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_devices

    # Handle restarting and resuming training
    if args.restart:
        print('Restarting training from scratch.')
        os.system('rm -rf {}'.format(args.model_path))

    if not os.path.isdir(args.model_path):
        os.system('mkdir -p {}'.format(args.model_path))
    else:
        print('Resuming training on model_path {}'.format(args.model_path))

    # Call training
    train(args)


Writing sh_train.py


In [33]:
#running the training script

#extract files from pretrained model
#!gzip -d '/content/IXI_HH_age_regression.tar.gz'
#!tar -xvf /content/IXI_HH_age_regression.tar

#assign path to pretrained model

#the issues I am running into may be because the tensorflow version used to 
#create the scripts is older than the current tensorflow version I am using...

#to upload local files to collab
#need to upload the demographic.csv file
from google.colab import files
files.upload()


!python /content/sh_train.py --model_path '/content/IXI_HH_age_regression'  --data_csv /content/demographic_HH.csv

Saving demographic_HH.csv to demographic_HH.csv
2020-10-11 20:12:20.496255: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "/content/sh_train.py", line 214, in <module>
    tf.logging.set_verbosity(tf.logging.ERROR)
AttributeError: module 'tensorflow' has no attribute 'logging'


In [17]:
#now that the model is trained 
#deploy aka output the results



Object `tensorflow_hub` not found.


In [63]:
#since tensorflow.contrib is no longer available i found the source code and will import the functions from there
path_to_predictor = "/content/predictor.py"
path_to_core_estimator_predictor = "/content/core_estimator_predictor.py"
path_to_contrib_estimator_predictor = "/content/contrib_estimator_predictor.py"
path_to_predictor_factories = "/content/predictor_factories.py"
path_to_saved_model_predictor = "/content/saved_model_predictor.py"
path_to_contrib_reader = "/content/contrib_reader.py"
import sys
import os
sys.path.append(os.path.abspath(path_to_predictor))
sys.path.append(os.path.abspath(path_to_core_estimator_predictor))
sys.path.append(os.path.abspath(path_to_contrib_estimator_predictor))
sys.path.append(os.path.abspath(path_to_predictor_factories))
sys.path.append(os.path.abspath(path_to_saved_model_predictor))
sys.path.append(os.path.abspath(path_to_contrib_reader))


import predictor
import predictor_factories
import core_estimator_predictor
import contrib_estimator_predictor
import saved_model_predictor
import contrib_reader


In [34]:
%%writefile predictor_factories.py


# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Factory functions for `Predictor`s."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import contrib_estimator_predictor
import core_estimator_predictor
import saved_model_predictor

#from tensorflow.contrib.learn.python.learn.estimators import estimator as contrib_estimator

from tensorflow.python.estimator import estimator as core_estimator


def from_contrib_estimator(estimator,
                           prediction_input_fn,
                           input_alternative_key=None,
                           output_alternative_key=None,
                           graph=None,
                           config=None):
  """Constructs a `Predictor` from a `tf.contrib.learn.Estimator`.
  Args:
    estimator: an instance of `tf.contrib.learn.Estimator`.
    prediction_input_fn: a function that takes no arguments and returns an
      instance of `InputFnOps`.
    input_alternative_key: Optional. Specify the input alternative used for
      prediction.
    output_alternative_key: Specify the output alternative used for
      prediction. Not needed for single-headed models but required for
      multi-headed models.
    graph: Optional. The Tensorflow `graph` in which prediction should be
      done.
    config: `ConfigProto` proto used to configure the session.
  Returns:
    An initialized `Predictor`.
  Raises:
    TypeError: if `estimator` is a core `Estimator` instead of a contrib
      `Estimator`.
  """
  if isinstance(estimator, core_estimator.Estimator):
    raise TypeError('Expected estimator to be of type '
                    'tf.contrib.learn.Estimator, but got type '
                    'tf.python.estimator.Estimator. You likely want to call '
                    'from_estimator.')
  return contrib_estimator_predictor.ContribEstimatorPredictor(
      estimator,
      prediction_input_fn,
      input_alternative_key=input_alternative_key,
      output_alternative_key=output_alternative_key,
      graph=graph,
      config=config)


def from_estimator(estimator,
                   serving_input_receiver_fn,
                   output_key=None,
                   graph=None,
                   config=None):
  """Constructs a `Predictor` from a `tf.python.estimator.Estimator`.
  Args:
    estimator: an instance of `learn.python.estimator.Estimator`.
    serving_input_receiver_fn: a function that takes no arguments and returns
      an instance of `ServingInputReceiver` compatible with `estimator`.
    output_key: Optional string specifying the export output to use. If
      `None`, then `DEFAULT_SERVING_SIGNATURE_DEF_KEY` is used.
    graph: Optional. The Tensorflow `graph` in which prediction should be
      done.
    config: `ConfigProto` proto used to configure the session.
  Returns:
    An initialized `Predictor`.
  Raises:
    TypeError: if `estimator` is a contrib `Estimator` instead of a core
      `Estimator`.
  """
  if isinstance(estimator, contrib_estimator.Estimator):
    raise TypeError('Expected estimator to be of type '
                    'tf.python.estimator.Estimator, but got type '
                    'tf.contrib.learn.Estimator. You likely want to call '
                    'from_contrib_estimator.')
  return core_estimator_predictor.CoreEstimatorPredictor(
      estimator,
      serving_input_receiver_fn,
      output_key=output_key,
      graph=graph,
      config=config)


def from_saved_model(export_dir,
                     signature_def_key=None,
                     signature_def=None,
                     input_names=None,
                     output_names=None,
                     tags=None,
                     graph=None,
                     config=None):
  """Constructs a `Predictor` from a `SavedModel` on disk.
  Args:
    export_dir: a path to a directory containing a `SavedModel`.
    signature_def_key: Optional string specifying the signature to use. If
      `None`, then `DEFAULT_SERVING_SIGNATURE_DEF_KEY` is used. Only one of
    `signature_def_key` and `signature_def`
    signature_def: A `SignatureDef` proto specifying the inputs and outputs
      for prediction. Only one of `signature_def_key` and `signature_def`
      should be specified.
      input_names: A dictionary mapping strings to `Tensor`s in the `SavedModel`
        that represent the input. The keys can be any string of the user's
        choosing.
      output_names: A dictionary mapping strings to `Tensor`s in the
        `SavedModel` that represent the output. The keys can be any string of
        the user's choosing.
    tags: Optional. Tags that will be used to retrieve the correct
      `SignatureDef`. Defaults to `DEFAULT_TAGS`.
    graph: Optional. The Tensorflow `graph` in which prediction should be
      done.
    config: `ConfigProto` proto used to configure the session.
  Returns:
    An initialized `Predictor`.
  Raises:
    ValueError: More than one of `signature_def_key` and `signature_def` is
      specified.
  """
  return saved_model_predictor.SavedModelPredictor(
      export_dir,
      signature_def_key=signature_def_key,
      signature_def=signature_def,
      input_names=input_names,
      output_names=output_names,
      tags=tags,
      graph=graph,
      config=config)

Overwriting predictor_factories.py


In [35]:
%%writefile predictor.py

# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Abstract base class for all predictors."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import six


@six.add_metaclass(abc.ABCMeta)
class Predictor(object):
  """Abstract base class for all predictors."""

  @property
  def graph(self):
    return self._graph

  @property
  def session(self):
    return self._session

  @property
  def feed_tensors(self):
    return self._feed_tensors

  @property
  def fetch_tensors(self):
    return self._fetch_tensors

  def __repr__(self):
    return '{} with feed tensors {} and fetch_tensors {}'.format(
        type(self).__name__, self._feed_tensors, self._fetch_tensors)

  def __call__(self, input_dict):
    """Returns predictions based on `input_dict`.
    Args:
      input_dict: a `dict` mapping strings to numpy arrays. These keys
        must match `self._feed_tensors.keys()`.
    Returns:
      A `dict` mapping strings to numpy arrays. The keys match
      `self.fetch_tensors.keys()`.
    Raises:
      ValueError: `input_dict` does not match `feed_tensors`.
    """
    # TODO(jamieas): make validation optional?
    input_keys = set(input_dict.keys())
    expected_keys = set(self.feed_tensors.keys())
    unexpected_keys = input_keys - expected_keys
    if unexpected_keys:
      raise ValueError(
          'Got unexpected keys in input_dict: {}\nexpected: {}'.format(
              unexpected_keys, expected_keys))

    feed_dict = {}
    for key in self.feed_tensors.keys():
      value = input_dict.get(key)
      if value is not None:
        feed_dict[self.feed_tensors[key]] = value
    return self._session.run(fetches=self.fetch_tensors, feed_dict=feed_dict)



Overwriting predictor.py


In [36]:
#make contrib_estimator_predictor script

%%writefile contrib_estimator_predictor.py

# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""A `Predictor constructed from a `tf.contrib.learn.Estimator`."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#from tensorflow.contrib.learn.python.learn.utils import saved_model_export_utils
import predictor_factories
import predictor
from tensorflow.python.framework import ops
from tensorflow.python.training import checkpoint_management
from tensorflow.python.training import monitored_session


class ContribEstimatorPredictor(predictor.Predictor):
  """A `Predictor constructed from a `tf.contrib.learn.Estimator`."""

  def __init__(self,
               estimator,
               prediction_input_fn,
               input_alternative_key=None,
               output_alternative_key=None,
               graph=None,
               config=None):
    """Initialize a `ContribEstimatorPredictor`.
    Args:
      estimator: an instance of `tf.contrib.learn.Estimator`.
      prediction_input_fn: a function that takes no arguments and returns an
        instance of `InputFnOps`.
      input_alternative_key: Optional. Specify the input alternative used for
        prediction.
      output_alternative_key: Specify the output alternative used for
        prediction. Not needed for single-headed models but required for
        multi-headed models.
      graph: Optional. The Tensorflow `graph` in which prediction should be
        done.
      config: `ConfigProto` proto used to configure the session.
    """
    self._graph = graph or ops.Graph()
    with self._graph.as_default():
      input_fn_ops = prediction_input_fn()
      # pylint: disable=protected-access
      model_fn_ops = estimator._get_predict_ops(input_fn_ops.features)
      # pylint: enable=protected-access
      checkpoint_path = checkpoint_management.latest_checkpoint(
          estimator.model_dir)
      self._session = monitored_session.MonitoredSession(
          session_creator=monitored_session.ChiefSessionCreator(
              config=config,
              checkpoint_filename_with_path=checkpoint_path))

    input_alternative_key = (
        input_alternative_key or
        saved_model_export_utils.DEFAULT_INPUT_ALTERNATIVE_KEY)
    input_alternatives, _ = saved_model_export_utils.get_input_alternatives(
        input_fn_ops)
    self._feed_tensors = input_alternatives[input_alternative_key]

    (output_alternatives,
     output_alternative_key) = saved_model_export_utils.get_output_alternatives(
         model_fn_ops, output_alternative_key)
    _, fetch_tensors = output_alternatives[output_alternative_key]
    self._fetch_tensors = fetch_tensors


Overwriting contrib_estimator_predictor.py


In [37]:
#make core_estimator_predictor
%%writefile core_estimator_predictor.py


# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""A `Predictor` constructed from an `learn.python.estimator.Estimator`."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import predictor_factories
import predictor
from tensorflow.python.estimator import model_fn
from tensorflow.python.framework import ops
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.training import monitored_session


def _get_signature_def(
    serving_input_receiver, estimator, output_key=None):
  """Construct a `SignatureDef` proto."""
  if output_key is None:
    output_key = signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY
  # pylint: disable=protected-access
  estimator_spec = estimator.model_fn(
      serving_input_receiver.features, None, model_fn.ModeKeys.PREDICT,
      estimator.config)
  # pylint: enable=protected-access
  export_outputs = estimator_spec.export_outputs
  export_output = export_outputs.get(output_key)
  if export_output is None:
    raise KeyError('output_key must be one of {}; got {}'.format(
        export_outputs.keys(), output_key))
  return export_output.as_signature_def(serving_input_receiver.receiver_tensors)


class CoreEstimatorPredictor(predictor.Predictor):
  """A `Predictor` constructed from an `learn.python.estimator.Estimator`."""

  def __init__(self,
               estimator,
               serving_input_receiver_fn,
               output_key=None,
               graph=None,
               config=None):
    """Initialize a `CoreEstimatorPredictor`.
    Args:
      estimator: an instance of `learn.python.estimator.Estimator`.
      serving_input_receiver_fn: a function that takes no arguments and returns
        an instance of `ServingInputReceiver` compatible with `estimator`.
      output_key: Optional string specifying the export output to use. If
        `None`, then `DEFAULT_SERVING_SIGNATURE_DEF_KEY` is used.
      graph: Optional. The Tensorflow `graph` in which prediction should be
        done.
      config: `ConfigProto` proto used to configure the session.
    """
    self._graph = graph or ops.Graph()
    with self._graph.as_default():
      serving_input_receiver = serving_input_receiver_fn()
      signature_def = _get_signature_def(
          serving_input_receiver, estimator, output_key)
      checkpoint_dir = estimator.model_dir
      self._session = monitored_session.MonitoredSession(
          session_creator=monitored_session.ChiefSessionCreator(
              config=config,
              checkpoint_dir=checkpoint_dir))

    feed_tensor_info = signature_def.inputs
    self._feed_tensors = {k: self._graph.get_tensor_by_name(v.name)
                          for k, v in feed_tensor_info.items()}
    fetch_tensor_info = signature_def.outputs
    self._fetch_tensors = {k: self._graph.get_tensor_by_name(v.name)
                           for k, v in fetch_tensor_info.items()}

Overwriting core_estimator_predictor.py


In [44]:
#make saved_model_predictor script
%%writefile saved_model_predictor.py


# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""A `Predictor` constructed from a `SavedModel`."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import logging

import predictor_factories
import predictor
#from tensorflow.contrib.saved_model.python.saved_model import reader
import contrib_reader as reader
from tensorflow.python.client import session
from tensorflow.python.framework import ops
from tensorflow.python.saved_model import loader
from tensorflow.python.saved_model import signature_constants


DEFAULT_TAGS = 'serve'

_DEFAULT_INPUT_ALTERNATIVE_FORMAT = 'default_input_alternative:{}'


def get_meta_graph_def(saved_model_dir, tags):
  """Gets `MetaGraphDef` from a directory containing a `SavedModel`.
  Returns the `MetaGraphDef` for the given tag-set and SavedModel directory.
  Args:
    saved_model_dir: Directory containing the SavedModel.
    tags: Comma separated list of tags used to identify the correct
      `MetaGraphDef`.
  Raises:
    ValueError: An error when the given tags cannot be found.
  Returns:
    A `MetaGraphDef` corresponding to the given tags.
  """
  saved_model = reader.read_saved_model(saved_model_dir)
  set_of_tags = set([tag.strip() for tag in tags.split(',')])
  for meta_graph_def in saved_model.meta_graphs:
    if set(meta_graph_def.meta_info_def.tags) == set_of_tags:
      return meta_graph_def
  raise ValueError('Could not find MetaGraphDef with tags {}'.format(tags))


def _get_signature_def(signature_def_key, export_dir, tags):
  """Construct a `SignatureDef` proto."""
  signature_def_key = (
      signature_def_key or
      signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY)

  metagraph_def = get_meta_graph_def(export_dir, tags)

  try:
    signature_def = metagraph_def.signature_def[signature_def_key]
  except KeyError as e:
    formatted_key = _DEFAULT_INPUT_ALTERNATIVE_FORMAT.format(
        signature_def_key)
    try:
      signature_def = metagraph_def.signature_def[formatted_key]
    except KeyError:
      raise ValueError(
          'Got signature_def_key "{}". Available signatures are {}. '
          'Original error:\n{}'.format(
              signature_def_key, list(metagraph_def.signature_def), e))
    logging.warning('Could not find signature def "%s". '
                    'Using "%s" instead', signature_def_key, formatted_key)
  return signature_def


def _check_signature_arguments(signature_def_key,
                               signature_def,
                               input_names,
                               output_names):
  """Validates signature arguments for `SavedModelPredictor`."""
  signature_def_key_specified = signature_def_key is not None
  signature_def_specified = signature_def is not None
  input_names_specified = input_names is not None
  output_names_specified = output_names is not None
  if input_names_specified != output_names_specified:
    raise ValueError(
        'input_names and output_names must both be specified or both be '
        'unspecified.'
    )

  if (signature_def_key_specified + signature_def_specified +
      input_names_specified > 1):
    raise ValueError(
        'You must specify at most one of signature_def_key OR signature_def OR'
        '(input_names AND output_names).'
    )


class SavedModelPredictor(predictor.Predictor):
  """A `Predictor` constructed from a `SavedModel`."""

  def __init__(self,
               export_dir,
               signature_def_key=None,
               signature_def=None,
               input_names=None,
               output_names=None,
               tags=None,
               graph=None,
               config=None):
    """Initialize a `CoreEstimatorPredictor`.
    Args:
      export_dir: a path to a directory containing a `SavedModel`.
      signature_def_key: Optional string specifying the signature to use. If
        `None`, then `DEFAULT_SERVING_SIGNATURE_DEF_KEY` is used. Only one of
        `signature_def_key` and `signature_def` should be specified.
      signature_def: A `SignatureDef` proto specifying the inputs and outputs
        for prediction. Only one of `signature_def_key` and `signature_def`
        should be specified.
      input_names: A dictionary mapping strings to `Tensor`s in the `SavedModel`
        that represent the input. The keys can be any string of the user's
        choosing.
      output_names: A dictionary mapping strings to `Tensor`s in the
        `SavedModel` that represent the output. The keys can be any string of
        the user's choosing.
      tags: Optional. Comma separated list of tags that will be used to retrieve
        the correct `SignatureDef`. Defaults to `DEFAULT_TAGS`.
      graph: Optional. The Tensorflow `graph` in which prediction should be
        done.
      config: `ConfigProto` proto used to configure the session.
    Raises:
      ValueError: If more than one of signature_def_key OR signature_def OR
        (input_names AND output_names) is specified.
    """
    _check_signature_arguments(
        signature_def_key, signature_def, input_names, output_names)
    tags = tags or DEFAULT_TAGS
    self._graph = graph or ops.Graph()

    with self._graph.as_default():
      self._session = session.Session(config=config)
      loader.load(self._session, tags.split(','), export_dir)

    if input_names is None:
      if signature_def is None:
        signature_def = _get_signature_def(signature_def_key, export_dir, tags)
      input_names = {k: v.name for k, v in signature_def.inputs.items()}
      output_names = {k: v.name for k, v in signature_def.outputs.items()}

    self._feed_tensors = {k: self._graph.get_tensor_by_name(v)
                          for k, v in input_names.items()}
    self._fetch_tensors = {k: self._graph.get_tensor_by_name(v)
                           for k, v in output_names.items()}


Overwriting saved_model_predictor.py


In [39]:
#make script for tensorflow.contrib.saved_model.python.saved_model.reader
%%writefile contrib_reader.py


# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""SavedModel functionality to read a SavedModel from disk."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

from google.protobuf import message
from google.protobuf import text_format
from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.lib.io import file_io
from tensorflow.python.saved_model import constants
from tensorflow.python.util import compat


def read_saved_model(saved_model_dir):
  """Reads the savedmodel.pb or savedmodel.pbtxt file containing `SavedModel`.
  Args:
    saved_model_dir: Directory containing the SavedModel file.
  Returns:
    A `SavedModel` protocol buffer.
  Raises:
    IOError: If the file does not exist, or cannot be successfully parsed.
  """
  # Build the path to the SavedModel in pbtxt format.
  path_to_pbtxt = os.path.join(
      compat.as_bytes(saved_model_dir),
      compat.as_bytes(constants.SAVED_MODEL_FILENAME_PBTXT))
  # Build the path to the SavedModel in pb format.
  path_to_pb = os.path.join(
      compat.as_bytes(saved_model_dir),
      compat.as_bytes(constants.SAVED_MODEL_FILENAME_PB))

  # Ensure that the SavedModel exists at either path.
  if not file_io.file_exists(path_to_pbtxt) and not file_io.file_exists(
      path_to_pb):
    raise IOError("SavedModel file does not exist at: %s" % saved_model_dir)

  # Parse the SavedModel protocol buffer.
  saved_model = saved_model_pb2.SavedModel()
  if file_io.file_exists(path_to_pb):
    try:
      file_content = file_io.FileIO(path_to_pb, "rb").read()
      saved_model.ParseFromString(file_content)
      return saved_model
    except message.DecodeError as e:
      raise IOError("Cannot parse file %s: %s." % (path_to_pb, str(e)))
  elif file_io.file_exists(path_to_pbtxt):
    try:
      file_content = file_io.FileIO(path_to_pbtxt, "rb").read()
      text_format.Merge(file_content.decode("utf-8"), saved_model)
      return saved_model
    except text_format.ParseError as e:
      raise IOError("Cannot parse file %s: %s." % (path_to_pbtxt, str(e)))
  else:
    raise IOError("SavedModel file does not exist at: %s/{%s|%s}" %
                  (saved_model_dir, constants.SAVED_MODEL_FILENAME_PBTXT,
                   constants.SAVED_MODEL_FILENAME_PB))


def get_saved_model_tag_sets(saved_model_dir):
  """Retrieves all the tag-sets available in the SavedModel.
  Args:
    saved_model_dir: Directory containing the SavedModel.
  Returns:
    String representation of all tag-sets in the SavedModel.
  """
  saved_model = read_saved_model(saved_model_dir)
  all_tags = []
  for meta_graph_def in saved_model.meta_graphs:
    all_tags.append(list(meta_graph_def.meta_info_def.tags))
  return all_tags


Overwriting contrib_reader.py


In [28]:
#had to edit to make it work with different tensorflow versions
%%writefile sh_deploy.py


from __future__ import division
from __future__ import print_function

import argparse
import os
import time

import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
#from tensorflow.contrib import predictor
import predictor_factories
import predictor

import tensorflow_hub

from dltk.io.augmentation import extract_random_example_array

from sh_abstract_reader import Reader

import reader 

tf.disable_v2_behavior()

READER_PARAMS = {'extract_examples': False}
N_VALIDATION_SUBJECTS = 28


def predict(args):
    # Read in the csv with the file names you would want to predict on
    file_names = pd.read_csv(
        args.csv,
        dtype=object,
        keep_default_na=False,
        na_values=[]).to_numpy()    #had to change to to_numpy

    # We trained on the first 4 subjects, so we predict on the rest
    file_names = file_names[-N_VALIDATION_SUBJECTS:]

    # From the model_path, parse the latest saved model and restore a
    # predictor from it
    export_dir = [os.path.join(args.model_path, o) for o in sorted(
        os.listdir(args.model_path)) if os.path.isdir(
        os.path.join(args.model_path, o)) and o.isdigit()][-1]

    print('Loading from {}'.format(export_dir))
    my_predictor = predictor_factories.from_saved_model(export_dir)

    # Iterate through the files, predict on the full volumes and compute a Dice
    # coefficient
    mae = []
    for output in reader.read_fn(file_references=file_names,
                          mode=tf.estimator.ModeKeys.EVAL,
                          params=READER_PARAMS):
        t0 = time.time()

        # Parse the read function output and add a dummy batch dimension as
        # required
        img = output['features']['x']
        lbl = output['labels']['y']
        test_id = output['img_id']

        # We know, that the training input shape of [64, 96, 96] will work with
        # our model strides, so we collect several crops of the test image and
        # average the predictions. Alternatively, we could pad or crop the input
        # to any shape that is compatible with the resolution scales of the
        # model:

        num_crop_predictions = 4
        crop_batch = extract_random_example_array(
            image_list=img,
            example_size=[64, 96, 96],
            n_examples=num_crop_predictions)

        y_ = my_predictor.session.run(
            fetches=my_predictor._fetch_tensors['logits'],
            feed_dict={my_predictor._feed_tensors['x']: crop_batch})

        # Average the predictions on the cropped test inputs:
        y_ = np.mean(y_)

        # Calculate the absolute error for this subject
        mae.append(np.abs(y_ - lbl))

        # Print outputs
        print('id={}; pred={:0.2f} yrs; true={:0.2f} yrs; run time={:0.2f} s; '
              ''.format(test_id, y_, lbl[0], time.time() - t0))
    print('mean absolute err={:0.3f} yrs'.format(np.mean(mae)))


if __name__ == '__main__':
    # Set up argument parser
    parser = argparse.ArgumentParser(
        description='IXI HH example age regression deploy script')
    parser.add_argument('--verbose', default=False, action='store_true')
    parser.add_argument('--cuda_devices', '-c', default='0')

    parser.add_argument('--model_path', '-p',
                        default='/content/IXI_HH_age_regression')
    parser.add_argument('--csv', default='/content/IXI_HH_age_regression/demographic_HH.csv')

    args = parser.parse_args()

    # Set verbosity
    if args.verbose:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
        tf.logging.set_verbosity(tf.logging.INFO)
    else:
        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
        tf.logging.set_verbosity(tf.logging.ERROR)

    # GPU allocation options
    os.environ["CUDA_VISIBLE_DEVICES"] = args.cuda_devices

    # Call training
    predict(args)


Overwriting sh_deploy.py


In [66]:
#run the deploy script
#this should work now....it does!!!!!

!python -u /content/sh_deploy.py --model_path '/content/IXI_HH_age_regression/'  --csv /content/demographic_HH.csv --verbose

2020-10-11 21:03:52.175370: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Instructions for updating:
non-resource variables are not supported in the long term
Loading from /content/IXI_HH_age_regression/1510857419
2020-10-11 21:03:53.761889: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-11 21:03:53.770552: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-10-11 21:03:53.770604: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5f6805a46636): /proc/driver/nvidia/version does not exist
2020-10-11 21:03:53.777649: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2200185000 Hz
2020-10-11 21:03:53.777890: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x